In [13]:
import numpy as np
from skimage import color
import matplotlib.pyplot as plt

# --- UTILITY FUNCTIONS ---
def clamp(val, lb, ub):
    return min(max(lb, val), ub)

def rgb_to_lab(rgb):
    rgb = np.array(rgb).reshape(1, 1, 3)
    lab = color.rgb2lab(rgb)
    return lab[0, 0]

def lab_to_rgb(lab):
    lab = np.array(lab).reshape(1, 1, 3)
    rgb = color.lab2rgb(lab)
    return rgb[0, 0]

# --- GLOBAL ---
filaments = {
    'Pearlescent Blue': {
        'rgb': [0.3137254901960784, 0.5058823529411764, 0.6901960784313725],
        'td': 3.9,   # Transmission Distance in mm
    },
    'Perfect Orange': {
        'rgb': [0.9490196078431372, 0.45098039215686275, 0.19215686274509805],
        'td': 3.0,
    },
    'Gun Metal Gray': {
        'rgb': [0.30980392156862746, 0.33725490196078434, 0.35294117647058826],
        'td': 0.7,
    },
    'Turquoise': {
        'rgb': [0.06274509803921569, 0.6, 0.5803921568627451],
        'td': 1.0,
    },
    'Hi-Flow Light Gray': {
        'rgb': [0.6470588235294118, 0.6509803921568628, 0.6392156862745098],
        'td': 0.5,
    },
    'Amethyst Violet': {
        'rgb': [0.5882352941176471, 0.27058823529411763, 0.48627450980392156],
        'td': 46.4,
    },
}

layer_thickness = 0.2  # mm per layer
base_color = [0.0, 0.0, 0.0] # [0.5, 0.5, 0.5]  # white background
target_color = [0.3, 0.7, 0.5]


# --- FUNCTIONS --- 

def barycentric_1d_on_segment(A: np.ndarray, B: np.ndarray, X: np.ndarray) -> float:
    """
    Computes the 1D barycentric coordinates u, v of point X on the line AB,
    such that X ≈ u * A + v * B.
    """
    AB = B - A
    denom = np.dot(AB, AB)
    if denom == 0:
        return 0.0  # A and B are the same point
    v = np.dot(X - A, AB) / denom
    u = 1.0 - v
    return u, v


def barycentric_2d_on_triangle(A: np.ndarray, B: np.ndarray, C: np.ndarray, X: np.ndarray) -> tuple[float, float, float]:
    """
    Computes the 2D barycentric coordinates (u, v, w) of point X with respect to triangle ABC,
    such that X ≈ u * A + v * B + w * C and u + v + w = 1.
    """
    v0 = B - A
    v1 = C - A
    v2 = X - A

    d00 = np.dot(v0, v0)
    d01 = np.dot(v0, v1)
    d11 = np.dot(v1, v1)
    d20 = np.dot(v2, v0)
    d21 = np.dot(v2, v1)

    denom = d00 * d11 - d01 * d01
    if denom == 0:
        return (1.0, 0.0, 0.0)  # Degenerate triangle

    v = (d11 * d20 - d01 * d21) / denom
    w = (d00 * d21 - d01 * d20) / denom
    u = 1.0 - v - w
    return (u, v, w)

def barycentric_3d_on_tetrahedron(A: np.ndarray, B: np.ndarray, C: np.ndarray, D: np.ndarray, X: np.ndarray) -> tuple[float, float, float, float]:
    """
    Computes 3D barycentric coordinates (u, v, w, t) of point X with respect to tetrahedron ABCD,
    such that X ≈ u*A + v*B + w*C + t*D and u + v + w + t = 1.
    """
    # Matrix of vectors relative to A
    M = np.column_stack((B - A, C - A, D - A))  # 3x3 matrix
    Y = X - A  # Vector from A to X

    try:
        coords = np.linalg.solve(M, Y)
        v, w, t = coords
        u = 1.0 - v - w - t
        return (u, v, w, t)
    except np.linalg.LinAlgError:
        # Degenerate tetrahedron (volume = 0)
        return (1.0, 0.0, 0.0, 0.0)
    
def barycentric_nd(points: list[np.ndarray], X: np.ndarray) -> list[float]:
    """
    Computes barycentric coordinates of point X with respect to a simplex defined by `points`.
    
    Arguments:
    - points: list of n+1 points (np.ndarray of same dimension), defining an n-dimensional simplex
    - X: point (np.ndarray) for which to compute barycentric coordinates
    
    Returns:
    - list of barycentric coordinates [b0, b1, ..., bn] such that sum(bi) = 1
    """
    P0 = points[0]
    Vs = [P - P0 for P in points[1:]]  # Edge vectors from P0
    V = np.column_stack(Vs)  # d x n matrix
    Y = X - P0

    try:
        coeffs = np.linalg.lstsq(V, Y, rcond=None)[0]  # solve V @ coeffs = Y
        bary_coords = [1.0 - np.sum(coeffs)] + list(coeffs)
        return bary_coords
    except np.linalg.LinAlgError:
        # Degenerate simplex (e.g., zero volume)
        return [1.0] + [0.0] * (len(points) - 1)




# --- MAIN ---
keys = list(filaments.keys())
color_a = filaments[keys[0]]['rgb']
color_a = np.array([1.0, 0.0, 0.0]) # red

color_b = filaments[keys[1]]['rgb']
color_b = np.array([0.0, 1.0, 0.0]) # green

target_color = np.array([1.0, 1.0, 0.0]) # yellow

result1 = barycentric_1d_on_segment(color_a, color_b, target_color)
result2 = barycentric_nd([color_a, color_b], target_color)
print(result1, result2)


(np.float64(0.5), np.float64(0.5)) [np.float64(0.5000000000000001), np.float64(0.4999999999999999)]
